<a href="https://colab.research.google.com/github/alexk2206/Data_Driven_Fantasy_Football/blob/dev/re_creation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import pandas as pd

In [2]:
file_path = 'https://raw.githubusercontent.com/alexk2206/Data_Driven_Fantasy_Football/refs/heads/dev/FantasyPros_2024_Overall_ADP_Rankings.csv'
df = pd.read_csv(file_path)#, on_bad_lines='skip')
df['POS'] = df['POS'].str.replace('\d+', '', regex=True)

df

,Rank,Player,Team,Bye,POS,ESPN,Sleeper,NFL,RTSports,FFC,AVG
0,1,Christian McCaffrey,SF,9,RB,1.0,1.0,1.0,1.0,NaN,1.0
1,2,CeeDee Lamb,DAL,7,WR,3.0,3.0,2.0,2.0,NaN,2.6
2,3,Tyreek Hill,MIA,6,WR,4.0,2.0,3.0,3.0,NaN,3.2
3,4,Bijan Robinson,ATL,12,RB,2.0,6.0,8.0,4.0,NaN,5.0
4,5,Breece Hall,NYJ,12,RB,5.0,8.0,7.0,5.0,NaN,5.4
...,...,...,...,...,...,...,...,...,...,...,...
943,944,Ryan Miller,TB,11,WR,NaN,NaN,894.0,NaN,NaN,894.0
944,945,Mitchell Tinsley,CIN,12,WR,NaN,NaN,895.0,NaN,NaN,895.0
945,946,La'Michael Pettway,NaN,NaN,TE,NaN,NaN,896.0,NaN,NaN,896.0
946,947,Anthony Gould,IND,14,WR,NaN,NaN,899.0,NaN,NaN,899.0


In [4]:
!pip install mip

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.3/15.3 MB 46.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 462.6/462.6 kB 22.2 MB/s eta 0:00:00
  Attempting uninstall: cffi
    Found existing installation: cffi 1.17.1
    Uninstalling cffi-1.17.1:
      Successfully uninstalled cffi-1.17.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
pygit2 1.18.0 requires cffi>=1.17.0, but you have cffi 1.15.1 which is incompatible.


In [5]:
from mip import Model, BINARY, maximize, xsum

# Initialisierung des Modells
model = Model("FantasyFootball", sense=maximize)

# Datenstrukturen (Beispielwerte)
players = df[['Player', 'POS']]
positions = {"QB", "RB", "WR", "TE", "K", "DST"}
weeks = list(range(1, 18))
gamma = {"QB": 2, "RB": 5, "WR": 6, "TE": 2, "K": 1, "DST": 2}
pos_limit = {"QB": 1, "RB": 2, "WR":2, "TE": 1, "K": 1, "DST": 1}

# Entscheidungsvariablen
y = {i: model.add_var(name=f"y_{i}", var_type=BINARY) for i in players}
x = {(i,t): model.add_var(name=f"x_{i}_{t}") for i in players for t in weeks}
z = {t: model.add_var(name=f"z_{t}", var_type=BINARY) for t in weeks}

# Zielfunktion (Beispielgewichte)
model.objective = (
    1.0 * xsum(f[i][t] * x[i,t] for i in players for t in weeks) +
    100.0 * xsum(z[t] for t in range(1,16)) +
    150.0 * xsum(z[t] for t in range(16,18))
)

# Constraints
for j in positions:
    # Mindestanzahl gedrafteter Spieler pro Position (1c)
    model += xsum(y[i] for i in players if pos[i] == j) >= gamma[j]

    # Positionslimits pro Woche (1d)
    for t in weeks:
        model += xsum(x[i,t] for i in players if pos[i] == j) <= pos_limit[j]

# Logische Verknüpfung Draft ↔ Aufstellung (1e)
for i in players:
    for t in weeks:
        model += x[i,t] <= y[i]

# Gewinnbedingung (1f)
for t in weeks:
    model += z[t] <= xsum(f[i][t] * x[i,t] for i in players) / beta[t]

# Bereits gedraftete Spieler (1g-1h)
for i in dm_players:
    model += y[i] == 1
for i in opp_players:
    model += y[i] == 0

# Optimierung
model.optimize()


NameError: name 'f' is not defined